In [73]:
import wandb
from wandb.apis.public import Api
from wandb.apis.public import Sweep, Run
from itertools import chain, islice  # using this for sake of memory
from collections import deque
import pandas as pd
import json
from tqdm import tqdm

In [83]:
# Utility funcs
def callable_attrs(obj, include_dunder=False):  # meh run it if you want to see whats callable
    names = []
    for name in dir(obj):
        if not include_dunder and name.startswith("__"):
            continue
        val = getattr(obj, name, None)
        if callable(val):
            names.append(name)
    return names
    
def get_metadata(run: Run):
    return run.metadata  # Just want to show its available

def slice_sweep_runs(sweep: Sweep, run_slice=None):
    """
    Returns a subset of sweep.runs based on a slice object with (runs_iterable, total_count)
    
    Example usage:
        slice_runs(sweep, slice(None, 10))   # first 10
        slice_runs(sweep, slice(10, 14))     # runs 10–13
        slice_runs(sweep, slice(-10, None))  # last 10
    """
    runs_iter = sweep.runs
    if run_slice is None: 
        return runs_iter, None

    # handle last N (negative start or stop)
    if run_slice.start is not None and run_slice.start < 0:
        n = abs(run_slice.start)
        tail = list(deque(runs_iter, maxlen=n))
        return tail, len(tail)

    if run_slice.stop is not None and run_slice.stop < 0:
        n = abs(run_slice.stop)
        tail = list(deque(runs_iter, maxlen=n))
        return tail, len(tail)

    # no None reg bounded slice
    start = run_slice.start or 0
    stop = run_slice.stop
    total = None if stop is None else max(0, stop - start)
    return islice(runs_iter, start, stop), total
    
def get_historical_columns(obj):
    if isinstance(obj, Sweep):
        run = next(obj.runs)
        return get_historical_columns(run)
    elif isinstance(obj, Run):
        columns = list(next(obj.scan_history()).keys())
        print(columns)
        return columns
    else:
        raise TypeError(f"Unsupported type: {type(obj)}")

def get_scanned_history(run: Run, keys=[], key_mode='include', drop_na_rows=True, pandas=False):
    if not keys: key_mode = 'exclude'

    # print(f"key mode is {key_mode}")
    historical_list = [] 
    for row in run.scan_history():  # Note can't use keys param because it only returns vals for cols with no nulls :( boo
        if key_mode == 'include':
            historical_list.append({k: v for k, v in row.items() if k in keys})
        else:  # assume 'exclude'
            historical_list.append({k: v for k, v in row.items() if k not in keys})
            
    if not pandas: return historical_list
    
    df = pd.DataFrame(historical_list)
    if drop_na_rows: df.dropna(axis=0, inplace=True)
    return df

def get_run_artifacts(run: Run):
    artifacts = []
    for art in run.logged_artifacts():
        artifacts.append(art)
    return artifacts

def get_run_config(run: Run):
    return {k: v.get("value") for k, v in json.loads(run.config).items() if k != "_wandb"}

def get_run_info(run: Run, include_config=True):
    return {
        **{"name": run.name, "id": run.id},
        **(get_run_config(run) if include_config else {})
    }
    
def get_run_summary(run: Run, include_config=True, exclude_states=['Failed', 'Crashed', 'Running'], include_system_metrics=False):
    item = get_run_info(run, include_config=include_config)

    history_keys = run.history_keys
    if history_keys.get('keys'):
        last_info = {}
        for key in [k for k in history_keys['keys'] if not k.startswith('system')]:
            entry = history_keys['keys'].get(key, {})
            type_value = entry.get('typeCounts', [])[0].get('type')
        
            if type_value != 'image-file': last_info[key] = entry.get('previousValue')  # add non-image metrics
        item.update(last_info)

    if include_system_metrics: item.update(json.loads(run.system_metrics))

    return item
    
def get_summary(sweep: Sweep, include_config=True, run_slice=None, pandas=True, exclude_states=['Failed', 'Crashed', 'Running'], include_system_metrics=False):
    # Note slice_sweep_runs allows us to slice which runs to use
    """
    Collects per-run summary information from a W&B sweep.

    Iterates through sweep and makes a summary record for each completed run. 
    Each record includes name, id, and metrics. (optionally includes: config and system stats)

    Args:
        sweep (Sweep): The W&B sweep object.
        include_config (bool): Whether to include run config metadata.
        run_slice (slice): Slice object specifying which runs to include.
            - `slice(None, 10)` → first 10 runs (lazy)
            - `slice(10, 20)` → runs 10–19 (lazy)
            - `slice(-10, None)` → last 10 runs (consumes sweep once) (sorry, don't know a differnt way!)
        pandas (bool): If True, return results as a pandas DataFrame; otherwise return a list of dicts.
        exclude_states (list): Run states to skip (e.g., 'Failed', 'Crashed', 'Running').
        include_system_metrics (bool): Include system metrics from `run.system_metrics`.

    Returns:
        pd.DataFrame | list[dict]: df or list of summaries
    """
    runs, total = slice_sweep_runs(sweep, run_slice)
    run_info = []
    
    for run in tqdm(runs, total=total, desc="Processing runs", leave=False):
        if run.state in exclude_states: continue  # Skip states that haven't finished
        run_info.append(get_run_summary(run, include_config=include_config, exclude_states=exclude_states, include_system_metrics=include_system_metrics))
    
    if not pandas: return run_info
    return pd.DataFrame(run_info)

# added doc string because way to many args ugh
def get_history(sweep: Sweep, keys=[], key_mode='include', use_sample=False, samples=100, include_config=True, run_slice=None, pandas=True, exclude_states=['Failed', 'Crashed', 'Running']):
    """
    Collects and optionally merges run histories from a W&B sweep.
    
    Iterates through sweep and grabs history real or sampled, merges with configuration info. 
    Returns a single pandas DataFrame if `pandas=True`, or separate lists of run and history.

    Args:
        sweep (Sweep): The W&B sweep object.
        keys (list): Columns to include or exclude from each run's history.
        key_mode (str): 'include' or 'exclude' behavior for `keys`.
        use_sample (bool): Whether to sample a subset of history points.
        samples (int): Number of samples to retrieve per run if sampling.
        include_config (bool): Whether to include run config metadata.
        run_slice (slice): Slice object specifying which runs to include.
            - `slice(None, 10)` → first 10 runs (lazy)
            - `slice(10, 20)` → runs 10–19 (lazy)
            - `slice(-10, None)` → last 10 runs (consumes sweep once) (sorry, don't know a differnt way!)
        pandas (bool): If True, return a combined DataFrame; otherwise, return raw lists.
        exclude_states (list): Run states to skip (e.g., 'Failed', 'Crashed', 'Running').

    Returns:
        pd.DataFrame | tuple[list[dict], list[list[dict]]]
        merged df    | run_info, run_history
    """
    runs, total = slice_sweep_runs(sweep, run_slice)
    
    run_info = []
    run_history = []

    for run in tqdm(runs, total=total, desc="Processing runs", leave=False):
        if run.state in exclude_states: continue  # Skip states that haven't finished

        run_info.append(get_run_info(run, include_config=include_config))  # Add run info. We will duplicate one to many for history

        if use_sample:  # we leave pandas false to just concat all together later, save some compute!
            run_history.append(run.history(samples=samples, pandas=False))
        else:
            run_history.append(get_scanned_history(run, keys=keys, key_mode=key_mode, pandas=False))
    
    if pandas: return pd.DataFrame(chain.from_iterable(({**info, **h} for h in hist) for info, hist in zip(run_info, run_history)))

    return run_info, run_history

In [33]:
sweep = Api().sweep("marcocassar-belmont-university/dl_experimentation-vae_train/sweeps/mho772yy")

#### Summary df

In [35]:
df = get_summary(sweep, include_config=True)

In [41]:
# Save to csv
df.to_csv(f"summary_{sweep.name}.csv", index=False)

#### History df
##### Important Note:
###### Wandb's run.history() returns sampled points from your run, while run.scan_history() returns a generator object with each log value.

###### I provide a boolean called `use_sample` to determine which to use. There is also a variety of other args to control what you want.

In [34]:
# If you want to get see what columns a run or sweep has use this function
cols = get_historical_columns(sweep)

['train_loss_ema', 'val_mae', 'reconstructions', 'step', 'kl_loss', 'val_kl_loss', 'val_loss', 'val_mse', 'val_psnr', '_timestamp', '_runtime', 'val_recon_loss', 'epoch', '_step', 'mu_mean', 'mu_std', 'beta*kl', 'beta', 'val_correlation', 'recon_loss', 'train_loss']


In [80]:
# Control Options | checkout the utility funcs if you want to see a few more hidden ones :)
include_config = True # includes the config hyperparamters from sweep
use_sample = False  # uses run.history() to sample values
samples = 100  # number of points sampled | used only if `use_sample` == True
key_mode = 'include' # or 'exclude' | controls method of filtering. if keys are empty will default to exclude
keys = []  # columns to filter by
run_slice = slice(0,5) # first 5 runs put None for all but watch out if not using sample and you have a lot of epochs or are a log monster

In [81]:
history_df = get_history(sweep, keys=keys, key_mode=key_mode, 
                         use_sample=use_sample, samples=samples, 
                         include_config=include_config, run_slice=slice(0,5))

In [82]:
history_df

,name,id,ema,epochs,groups,use_bn,dropout,use_skips,activation,batch_size,...,val_recon_loss,epoch,_step,mu_mean,mu_std,beta*kl,beta,val_correlation,recon_loss,train_loss
0,dauntless-sweep-1,3a2z4qrq,0.97,25,4,True,0.402686,True,ReLU,256,...,NaN,NaN,0,NaN,NaN,0.218256,0.255562,NaN,625.060242,625.278503
1,dauntless-sweep-1,3a2z4qrq,0.97,25,4,True,0.402686,True,ReLU,256,...,NaN,NaN,1,NaN,NaN,0.255105,0.255562,NaN,528.210205,528.465332
2,dauntless-sweep-1,3a2z4qrq,0.97,25,4,True,0.402686,True,ReLU,256,...,NaN,NaN,2,NaN,NaN,0.325361,0.255562,NaN,458.326019,458.651367
3,dauntless-sweep-1,3a2z4qrq,0.97,25,4,True,0.402686,True,ReLU,256,...,NaN,NaN,3,NaN,NaN,0.427037,0.255562,NaN,410.284790,410.711823
4,dauntless-sweep-1,3a2z4qrq,0.97,25,4,True,0.402686,True,ReLU,256,...,NaN,NaN,4,NaN,NaN,0.454300,0.255562,NaN,366.298767,366.753082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35370,valiant-sweep-8,hqzumrt1,0.97,25,2,True,0.555190,True,LeakyReLU,256,...,NaN,NaN,5900,NaN,NaN,13.262024,0.463046,NaN,82.046692,95.308716
35371,valiant-sweep-8,hqzumrt1,0.97,25,2,True,0.555190,True,LeakyReLU,256,...,NaN,NaN,5901,NaN,NaN,13.243394,0.463046,NaN,80.518745,93.762138
35372,valiant-sweep-8,hqzumrt1,0.97,25,2,True,0.555190,True,LeakyReLU,256,...,NaN,NaN,5902,NaN,NaN,13.237176,0.463046,NaN,82.256004,95.493179
35373,valiant-sweep-8,hqzumrt1,0.97,25,2,True,0.555190,True,LeakyReLU,256,...,0.095338,25.0,5903,-0.001205,0.624191,NaN,NaN,0.937433,NaN,NaN


In [50]:
history_df.to_csv(f"history_{sweep.name}.csv", index=False)

### Now go play with the data!

You can explore your results directly in the notebook or export them to CSV files for later analysis.  
Or try to check them out in **[SweepViewer](https://sweepviewer.streamlit.app/)**!

In [69]:
# If you want to checkout the artifacts available to a run try below
run = next(iter(sweep.runs))
artifacts = get_run_artifacts(run)

In [ ]:
# You can use `callable_attrs` to see whats available | I recommend just using `.download()`
# callable_attrs(artifacts[0])
# print(artifacts[0].download.__doc__)  # check the doc string for more too...
artifacts[0].download()

In [ ]:
# Play zone